In [4]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

geo_data = pd.read_csv('distance_to_lake.csv')
feature_data = geo_data[['num','d_water','d_park',
       'd_lm','TL1','Tmin','Tmax']].dropna()
feature_data['sqrt_dw']=np.sqrt(feature_data['d_water'])
feature_data['sqrt_dp']=np.sqrt(feature_data['d_park'])
trainX=feature_data[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
trainy=feature_data[['num']].astype(float)

estimators = [10, 20, 50, 100, 200]
scores = []
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(trainX,trainy)


#for n_est in estimators:
#    clf = RandomForestRegressor(n_estimators=n_est, random_state=42, cv=5,max_depth = 5)
#    score = np.mean(cross_val_score(clf, trainX, trainy))
#    scores.append(score)
#    print(n_est)

In [14]:
latmin = min(geo_data.lat)
latmax = max(geo_data.lat)
lonmin = min(geo_data.lon)
lonmax = max(geo_data.lon)

grd = np.indices((51,51)).T.reshape(-1,2)
grd = grd.astype(float)
grd[:,1] = (1/50.)*grd[:,1]*(lonmax - lonmin) + lonmin
grd[:,0] = (1/50.)*grd[:,0]*(latmax - latmin) + latmin
xx = np.linspace(lonmin,lonmax,51)
yy = np.linspace(latmin,latmax,51)

geogrid = pd.read_csv('gridlake.csv')
testX = pd.DataFrame(geogrid['d_lm'])
testX['TL1'] = 75.18
testX['sqrt_dw']=np.sqrt(geogrid['d_water'])
testX['sqrt_dp']=np.sqrt(geogrid['d_park'])
testX['Tmax'] = 86.63 
testX['Tmin'] = 65.18
testX = testX[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(trainX,trainy)

YY = rfr.predict(testX)
testY = YY.reshape(51,51).transpose()

/Users/ianbakst/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [15]:
import branca
import folium
from folium import plugins
from shapely.geometry import Point, Polygon
import geojsoncontour
import matplotlib.pyplot as plt


colors = ['darkgreen','green','limegreen','yellow','gold','orange','red','darkred']
vmin = 0
vmax = 50
levels = len(colors)
cm = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

cfig = plt.contourf(xx, yy, testY, levels, alpha=1.0, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
plt.savefig('map.png',dpi=300)
geojson = geojsoncontour.contourf_to_geojson(contourf=cfig,
                                             min_angle_deg=0.0,
                                             ndigits=5,stroke_width=0.5,
                                             fill_opacity=1.0)

geomap = folium.Map(location=[41.89, -87.64], zoom_start=13, tiles="stamenterrain")
folium.GeoJson(geojson,style_function=lambda x:{'color': x['properties']['stroke'],
                   'weight': x['properties']['stroke-width'],
                   'fillColor': x['properties']['fill'],
                   'opacity':  1.0}).add_to(geomap)
cm.caption = 'Mosquito Activity'
geomap.add_child(cm)
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.save(f'rf_activity_map.html')
plt.close()

In [11]:
trainX

,d_lm,TL1,Tmin,Tmax,sqrt_dw,sqrt_dp
0,0.030931,68.0,56.0,90.0,0.129137,0.027096
1,0.162091,68.0,56.0,90.0,0.136348,0.063283
2,0.030931,68.0,56.0,90.0,0.129137,0.027096
3,0.071573,68.0,56.0,90.0,0.011400,0.000000
4,0.098231,68.0,56.0,90.0,0.080034,0.014381
...,...,...,...,...,...,...
24765,0.162091,65.0,61.0,93.0,0.136348,0.063283
24766,0.060025,65.0,61.0,93.0,0.098772,0.031207
24767,0.034664,65.0,61.0,93.0,0.022931,0.056370
24768,0.128273,65.0,61.0,93.0,0.156301,0.053425


In [121]:
import fiona
import numpy as np
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import pandas as pd
import geopandas as gpd
import pickle

multipol = fiona.open("geography/city_limits/geo_export_0001e424-08ad-4bde-885b-0b5ac036a564.shp")
multi = multipol.next() # only one feature in the shapefile

geogrid = pd.read_csv('gridlake.csv')
testX = pd.DataFrame(geogrid['d_lm'])
testX['TL1'] = 75.18
testX['sqrt_dw']=np.sqrt(geogrid['d_water'])
testX['sqrt_dp']=np.sqrt(geogrid['d_park'])
testX['Tmax'] = 86.63 
testX['Tmin'] = 65.18
testX = testX[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
lin = pickle.load(open('lin_model.sav','rb'))
testY = lin.predict(testX)
smallY = testY.reshape(51,51).transpose()
Y = ndimage.zoom(smallY, 5)

grd3 = np.indices((len(Y),len(Y))).T.reshape(-1,2)
grd3 = grd3.astype(float)
grd3[:,1] = (1/float(len(Y)-1))*grd3[:,1]*(lonmax - lonmin) + lonmin
grd3[:,0] = (1/float(len(Y)-1))*grd3[:,0]*(latmax - latmin) + latmin
x3=np.linspace(lonmin,lonmax,len(Y))
y3=np.linspace(latmin,latmax,len(Y))
YY = Y.T.reshape(-1,1)
nn = np.ones(shape = (len(YY),1))
#plt.contourf(x3,y3,Y)
crs = {'init': 'epsg:4326'}
geometry = [Point(xy) for xy in zip(grd3[:,1],grd3[:,0])]
points = gpd.GeoDataFrame(crs = crs, geometry = geometry)
for i,pt in points.iterrows():
    point = shape(pt['geometry'])
    if not(point.within(shape(multi['geometry']))):
        YY[i] = np.nan
        nn[i] = np.nan           
bigY = YY.reshape(len(Y),len(Y)).transpose()
NN = nn.reshape(len(Y),len(Y)).transpose()


/Users/ianbakst/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  if __name__ == '__main__':


In [104]:
testX = pd.DataFrame(geogrid['d_lm'])
testX['TL1'] = 75.18
testX['sqrt_dw']=np.sqrt(geogrid['d_water'])
testX['sqrt_dp']=np.sqrt(geogrid['d_park'])
testX['Tmax'] = 86.63 
testX['Tmin'] = 65.18
testX = testX[['d_lm','TL1','Tmin','Tmax','sqrt_dw','sqrt_dp']]
lin = pickle.load(open('lin_model.sav','rb'))
testY = lin.predict(testX)
smallY = testY.reshape(51,51).transpose()
Y = ndimage.zoom(smallY, 3)
x3=np.linspace(lonmin,lonmax,len(AA))
y3=np.linspace(latmin,latmax,len(AA))
np.shape(Y)

(153, 153)

In [113]:
import branca
import geojsoncontour
import folium
from folium import plugins
from folium.plugins import FloatImage
from scipy import ndimage
colors = ['darkgreen','green','limegreen','yellow','gold','orange','red','darkred']
vmin = 0
vmax = 50
levels = len(colors)

#AA = ndimage.zoom(testY, 3)
#x3=np.linspace(lonmin,lonmax,len(AA))
#y3=np.linspace(latmin,latmax,len(AA))


cm = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)

cfig = plt.contourf(x3, y3, bigY, levels, alpha=1.0, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
plt.savefig('map.png',dpi=300)
geojson = geojsoncontour.contourf_to_geojson(contourf=cfig,
                                             min_angle_deg=0.0,
                                             ndigits=5,stroke_width=0.5,
                                             fill_opacity=1.0)

geomap = folium.Map(location=[41.89, -87.64], zoom_start=13, tiles="stamenterrain")
folium.GeoJson(geojson,style_function=lambda x:{'color': x['properties']['stroke'],
                   'weight': x['properties']['stroke-width'],
                   'fillColor': x['properties']['fill'],
                   'fillOpacity':  0.65}).add_to(geomap)
FloatImage('scale.png', bottom=40, left=65).add_to(geomap)
#plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.save(f'activity_map.html')
plt.close()

In [123]:
np.savetxt("nans.csv", NN, delimiter=",")

In [125]:
NNN = np.genfromtxt("nans.csv",delimiter = ',')

In [127]:
multY = np.multiply(Y,NN)

In [129]:
cfig = plt.contourf(x3, y3, multY, levels, alpha=1.0, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
plt.savefig('map.png',dpi=300)
geojson = geojsoncontour.contourf_to_geojson(contourf=cfig,
                                             min_angle_deg=0.0,
                                             ndigits=5,stroke_width=0.5,
                                             fill_opacity=1.0)

geomap = folium.Map(location=[41.89, -87.64], zoom_start=13, tiles="stamenterrain")
folium.GeoJson(geojson,style_function=lambda x:{'color': x['properties']['stroke'],
                   'weight': x['properties']['stroke-width'],
                   'fillColor': x['properties']['fill'],
                   'fillOpacity':  0.65}).add_to(geomap)
#FloatImage('scale.png', bottom=40, left=65).add_to(geomap)
#plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.save(f'activity_map.html')
plt.close()

In [130]:
np.shape(NN)

(255, 255)